In [1]:
from pymongo import MongoClient
from datetime import datetime

In [28]:
import pandas as pd
import numpy as np

In [2]:
client = MongoClient('127.0.0.1', 27017)
db = client.top1000
train = db.tweets

In [3]:
train.find_one(filter={"user.screen_name":"Twitch"})

{'_id': ObjectId('5ad68a1b7ce0e03570faa655'),
 'contributors': None,
 'coordinates': None,
 'created_at': 'Tue Apr 17 18:06:05 +0000 2018',
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': 'twitter.com/i/web/status/9…',
    'expanded_url': 'https://twitter.com/i/web/status/986304809862094849',
    'indices': [116, 139],
    'url': 'https://t.co/Dt3gUOQJhw'}],
  'user_mentions': []},
 'favorite_count': 1888,
 'favorited': False,
 'geo': None,
 'id': 986304809862094849,
 'id_str': '986304809862094849',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'possibly_sensitive': False,
 'retweet_count': 500,
 'retweeted': False,
 'source': '<a href="http://www.spredfast.com" rel="nofollow">Spredfast app</a>',
 'text': 'Our new Channel Analytics page is now live! See your Stats and Revenue combined

In [4]:
all_tweets=train.find(filter={"lang":"en"},projection=["text","retweet_count","user.screen_name","retweeted_status","created_at","entities"])

In [5]:
tweets_list=[{"text":x["text"],"retweet_count":x["retweet_count"],"screen_name":x["user"]["screen_name"],
                "retweeted":True if "retweeted_status" in x else False, "created_at":x["created_at"],"entities":x["entities"]} 
               for x in all_tweets]

In [8]:
df=pd.DataFrame(tweets_list)

In [13]:
df.to_json("all_original_tweets_top1000.json")

In [21]:
start_time = "Thu Apr 10 00:00:00 +0000 2018"
start_time = datetime.strptime(start_time, "%a %b %d %H:%M:%S +0000 %Y")

In [15]:
df["time"]=df["created_at"].apply(lambda x:datetime.strptime(x, "%a %b %d %H:%M:%S +0000 %Y"))

In [22]:
train_df=all_df[all_df["time"]<start_time]

In [32]:
test_df=all_df[all_df["time"]>=start_time]

In [33]:
len(train_df)+len(test_df)==len(all_df)

True

In [24]:
_/len(df)

0.7854594906146993

In [26]:
all_df=df
df=train_df

In [29]:
median_s=df.groupby("screen_name").apply(lambda x:np.median(x["retweet_count"]))
median_df=pd.DataFrame({"screen_name":median_s.index.values,"retweet_median":median_s.values})
print(median_df[median_df.retweet_median>0].describe())
merged_df=pd.merge(df,median_df,on="screen_name",how="left")

       retweet_median
count      884.000000
mean      1684.278846
std      11446.678295
min          0.500000
25%         35.000000
50%        150.750000
75%        556.500000
max     273362.500000


In [40]:
merged_test_df=pd.merge(test_df,median_df,on="screen_name",how="left")

In [44]:
merged_test_df.retweet_median.count()

80218

In [48]:
merged_test_df=merged_test_df.dropna()

In [47]:
merged_test_df

80218

In [51]:
merged_df.to_json("train_original_tweets_top1000.json")

In [52]:
merged_test_df.to_json("test_original_tweets_top1000.json")